# Mean Average Precision (mAP) Explained and PyTorch Implementation
- https://www.youtube.com/watch?v=FppOzcDvaDI

Understand and implement the most common metric used in Deep learning to evaluate object detection models

1. Get all bounding box predictions on out test set
2. Get False Positive and True Positive
3. Calculate Precision and Recall
4. Plot precision vs recall graph
5. Precision of class is the area under the graph
6. Average all the precision for a class 
7. Select new iou threshold

- mAP,0.5,0.05,0.95
- 0.5 to 0.95 iou threshold  with step size of 0.05

In [6]:
import torch
from collections import Counter
from iou import intersection_over_union

In [30]:
def mean_average_precision(pred_boxes, 
                       true_boxes,
                       iou_threshold=0.5,   
                       box_format="corners",
                       num_classes=20):
    """
        pred_boxes (list) = [[train_idx, class_pred, prob_score, x1, y1, x2, y2], ...]
    """
    average_precisions = []
    epsilon = 1e-6
    
    for c in range(num_classes):
        """
            To store pred boxes and true boxes
            for a specific class
        """
        detections = []
        ground_truths = []
        
        for pred_box in pred_boxes:
            if pred_box[1] == c:
                detections.append(pred_box)
        
        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)
        
        """
            img0 can have 3 bboxes
            img1 can have 5 bboxes
            gt[0] gives the key , 
            look in counter docs
            list1 = ['x','y','z','x','x','x','y', 'z']
            Counter({'x': 4, 'y': 2, 'z': 2})
            for our case instead of x and y we will be having train_idx
            amount_of_bboxes = {0:3, 1:5}
        """
        amount_of_bboxes = Counter([gt[0] for gt in ground_truths])
        
        for key, val in amount_of_bboxes.items():
              amount_of_bboxes[key] = torch.zeros(val)
        
        """
            High confidence 1st
        """
        detections.sort(key= lambda x: x[2], reverse=True)
        TP = torch.zeros(len(detections))
        FP = torch.zeros(len(detections))
        total_true_bboxes = len(ground_truths)
        
        for detection_idx, detection in enumerate(detections):
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]  # taking all the ground truth matching the training idx of the detection.
            num_gts = len(ground_truth_img)  # no. of targets. # NOT USED
            best_iou = 0
            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(torch.tensor(detection[3:]),
                                             torch.tensor(gt[3:]),
                                              box_format=box_format,
                                             )
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx  # for which target the detection best matched
            
            if best_iou > iou_threshold:

                if amount_of_bboxes[detection[0]][best_gt_idx] == 0:
                    TP[detection_idx] = 1
                    amount_of_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1  
                    """
                        *cONFUSION*
                        Make false for the extra detection 
                    """
            else:
                    FP[detection_idx] = 1
                    
        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        
        precisions = torch.cat((torch.tensor([1]), precisions))  # (0, 1)
        recalls = torch.cat((torch.tensor([0]), recalls))  # (0, 1)
        average_precisions.append(torch.trapz(precisions, recalls))
    
    return sum(average_precisions) / len(average_precisions) 
#     pass 

In [28]:
import unittest
import torch

class TestMeanAveragePrecision(unittest.TestCase):
    def setUp(self):
        # test cases we want to run
        self.t1_preds = [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t1_targets = [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t1_correct_mAP = 1

        self.t2_preds = [
            [1, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t2_targets = [
            [1, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t2_correct_mAP = 1

        self.t3_preds = [
            [0, 1, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 1, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 1, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t3_targets = [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t3_correct_mAP = 0

        self.t4_preds = [
            [0, 0, 0.9, 0.15, 0.25, 0.1, 0.1],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]

        self.t4_targets = [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2],
        ]
        self.t4_correct_mAP = 5 / 18

        self.epsilon = 1e-4

    def test_all_correct_one_class(self):
        mean_avg_prec = mean_average_precision(
            self.t1_preds,
            self.t1_targets,
            iou_threshold=0.5,
            box_format="midpoint",
            num_classes=1,
        )
        self.assertTrue(abs(self.t1_correct_mAP - mean_avg_prec) < self.epsilon)

    def test_all_correct_batch(self):
        mean_avg_prec = mean_average_precision(
            self.t2_preds,
            self.t2_targets,
            iou_threshold=0.5,
            box_format="midpoint",
            num_classes=1,
        )
        self.assertTrue(abs(self.t2_correct_mAP - mean_avg_prec) < self.epsilon)

    def test_all_wrong_class(self):
        mean_avg_prec = mean_average_precision(
            self.t3_preds,
            self.t3_targets,
            iou_threshold=0.5,
            box_format="midpoint",
            num_classes=2,
        )
        self.assertTrue(abs(self.t3_correct_mAP - mean_avg_prec) < self.epsilon)

    def test_one_inaccurate_box(self):
        mean_avg_prec = mean_average_precision(
            self.t4_preds,
            self.t4_targets,
            iou_threshold=0.5,
            box_format="midpoint",
            num_classes=1,
        )
        self.assertTrue(abs(self.t4_correct_mAP - mean_avg_prec) < self.epsilon)

    def test_all_wrong_class(self):
        mean_avg_prec = mean_average_precision(
            self.t3_preds,
            self.t3_targets,
            iou_threshold=0.5,
            box_format="midpoint",
            num_classes=2,
        )
        self.assertTrue(abs(self.t3_correct_mAP - mean_avg_prec) < self.epsilon)




In [31]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_all_correct_batch (__main__.TestMeanAveragePrecision) ... ok
test_all_correct_one_class (__main__.TestMeanAveragePrecision) ... ok
test_all_wrong_class (__main__.TestMeanAveragePrecision) ... ok
test_one_inaccurate_box (__main__.TestMeanAveragePrecision) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.018s

OK
